### 녹음을 위한 패키지 설치

In [1]:
!pip install sounddevice
!pip install soundfile

#### 음성 녹음 test

In [3]:
import time
import queue, os, threading
import sounddevice as sd
import soundfile as sf
from scipy.io.wavfile import write

q = queue.Queue()
recorder = False
recording = False

def complicated_record():
    with sf.SoundFile("test.wav", mode='w', samplerate=16000, subtype='PCM_16', channels=1) as file:
        with sd.InputStream(samplerate=16000, dtype='int16', channels=1, callback=complicated_save):
            while recording:
                file.write(q.get())
        
def complicated_save(indata, frames, time, status):
    q.put(indata.copy())
    
def start():
    global recorder
    global recording
    recording = True
    recorder = threading.Thread(target=complicated_record)
    print('start recording')
    recorder.start()
    
def stop():
    global recorder
    global recording
    recording = False
    recorder.join()
    print('stop recording')
    
start()
time.sleep(5)
stop()

start recording
stop recording


### 1단계: 카카오 개발자 페이지 접속하기
1. 카카오 디벨로퍼스에 접속합니다.(https://developers.kakao.com)
2. 오른쪽 상단의 로그인 버튼을 클릭해 카카오 계정으로 로그인 합니다. *카카오 계정이 없는 경우 새롭게 계정을 생성합니다.

### 2단계: 내 애플리케이션 만들기
1. 상단 메뉴에서 내 애플리케이션을 클릭합니다.
2. 서비스 이용 동의 페이지가 나타나는 경우 필수 항목에 동의하고, 개발자 이름을 입력한 다음 회원가입 버튼을 클 릭해 다음 단계로 넘어갑니다. (기존에 서비스 이용 동의한 경우 표시되지 않을 수 있습니다.)
3. 전체 애플리케이션 페이지가 나타나면, 애플리케이션 추가하기를 클릭합니다.
4. 앱 아이콘(로고)을 업로드하고, 앱 이름, 회사 이름을 작성합니다.
5. 저장 버튼을 클릭합니다.
6. REST API 키 복사
7. 왼쪽 메뉴중 음성 메뉴를 선택후 상태를 On으로 바꿔야 됨

#### STT 테스트

In [4]:
import os
import requests
import json
kakao_speech_url = "https://kakaoi-newtone-openapi.kakao.com/v1/recognize"

rest_api_key = os.getenv("KAKAO_API_KEY")

headers = {
    "Content-Type": "application/octet-stream",
    "X-DSS-Service": "DICTATION",
    "Authorization": "KakaoAK " + rest_api_key,
}

with open('test.wav', 'rb') as fp:
    audio = fp.read()

res = requests.post(kakao_speech_url, headers=headers, data=audio)
print(res.text)

------newtone84jJYf3cRFwFqLdw
Content-Type: application/json; charset=UTF-8

{"type":"beginPointDetection","value":"BPD"}
------newtone84jJYf3cRFwFqLdw
Content-Type: application/json; charset=UTF-8

{"type":"partialResult","value":"카카오"}
------newtone84jJYf3cRFwFqLdw
Content-Type: application/json; charset=UTF-8

{"type":"partialResult","value":"카카오 비"}
------newtone84jJYf3cRFwFqLdw
Content-Type: application/json; charset=UTF-8

{"type":"endPointDetection","value":"EPD"}
------newtone84jJYf3cRFwFqLdw
Content-Type: application/json; charset=UTF-8

{"type":"partialResult","value":"카카오 bts"}
------newtone84jJYf3cRFwFqLdw
Content-Type: application/json; charset=UTF-8

{"type":"partialResult","value":"카카오 비티에스 연습죠"}
------newtone84jJYf3cRFwFqLdw
Content-Type: application/json; charset=UTF-8
Speech-Length: 4

{"type":"finalResult","value":"카카오 비티에스 연습죠","nBest":[{"value":"카카오 비티에스 연습죠","score":40},{"value":"카카오 bts 연습죠","score":0},{"value":"카카오야 비티에스 연습죠","score":0}],"voiceProfile":{"registe

#### finalResult 추출 방법

In [5]:
result_json_string = res.text[res.text.index('{"type":"finalResult"'):res.text.rindex('}')+1]
result = json.loads(result_json_string)
# print(result)
print(result['value'])

카카오 비티에스 연습죠


#### TTS 테스트

In [11]:
import os, requests, json, wave
from IPython.display import Audio
from IPython.display import display

url = "https://kakaoi-newtone-openapi.kakao.com/v1/synthesize"
key = os.getenv("KAKAO_API_KEY")
headers = {
    "Content-Type": "application/xml",
    "Authorization": "KakaoAK " + key,
}

In [12]:
text = "안녕하세요. 제 이름은 카카오 음성합성 API 입니다. 만나서 반가워요. ㅋㅋㅋ"

data = f"<speak>{text}</speak>".encode('utf-8').decode('latin1')
res = requests.post(url, headers=headers, data=data)

with open('hello_kakao.wav', 'wb') as f:
    f.write(res.content)
    
wn = Audio('hello_kakao.wav', autoplay=True)
display(wn)

In [14]:
data = '''
<speak>
  <voice name="WOMAN_READ_CALM"> 지금은 여성 차분한 낭독체입니다.</voice>
  <voice name="MAN_READ_CALM"> 지금은 여성 차분한 낭독체입니다.</voice>
  <voice name="WOMAN_DIALOG_BRIGHT"> 안녕하세요. 여성 밝은 대화체예요.</voice>
  <voice name="MAN_DIALOG_BRIGHT"> 안녕하세요. 남성 밝은 대화체예요.</voice>
</speak>'''.encode('utf-8').decode('latin1')

res = requests.post(url, headers=headers, data=data)
with open('voice.wav', 'wb') as f:
    f.write(res.content)
    
wn = Audio('voice.wav', autoplay=True)
display(wn)

In [15]:
data = '''
<speak>
  <prosody rate="fast" volume="loud">안녕하세요. 반가워요.</prosody>
  <prosody rate="slow" volume="soft">안녕하세요. 반가워요.</prosody>
</speak>'''.encode('utf-8').decode('latin1')

res = requests.post(url, headers=headers, data=data)
with open('prosody.wav', 'wb') as f:
    f.write(res.content)
    
wn = Audio('prosody.wav', autoplay=True)
display(wn)

In [16]:
data = '''
<speak>
  첫 번째 문장입니다.
  <break/>두 번째 문장입니다. 옵션을 안준 경우 기본 셋팅 150입니다.
  세 번째 문장입니다.
  <break time='150ms'/>네 번째 문장입니다. 앞이랑 똑같죠?
  <break time='1500ms'/>다섯 번째 문장입니다. 길게 쉬죠?
  ㄹㅋㅌㄷㅊㄹ
  <break time='1500ms'/>여섯 번째 문장입니다. 앞에 문장이 실패해서 쉬지 않습니다.
</speak>'''.encode('utf-8').decode('latin1')

res = requests.post(url, headers=headers, data=data)
with open('break.wav', 'wb') as f:
    f.write(res.content)
    
wn = Audio('break.wav', autoplay=True)
display(wn)

In [18]:
data = '''
<speak>
  <kakao:effect tone="friendly">안녕하세요. 반가워요. 원래는 존댓말인데 반말로 바꿔보겠습니다.
  <kakao:effect tone="default">잘 지내요? 여긴 존댓말 구역이에요.</kakao:effect>
  <prosody>저도 잘 지내요.</prosody>
  </kakao:effect>
</speak>'''.encode('utf-8').decode('latin1')

res = requests.post(url, headers=headers, data=data)
with open('friendly.wav', 'wb') as f:
    f.write(res.content)
    
wn = Audio('friendly.wav', autoplay=True)
display(wn)